In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

PROJECT_JSON = kraft.read_json("../project.json")

PATH = make_path_dict(PROJECT_JSON)

In [ ]:
w = pd.read_csv(PATH["w.tsv"], sep="\t", index_col=0)

w.columns.name = "Factor"

h = pd.read_csv(PATH["h.tsv"], sep="\t", index_col=0)

h.columns.name = PROJECT_JSON["sample_alias"]

In [ ]:
for feature_type, w_or_h, dataframe, apply_axis, drop_axis in (
    ("feature", "w", w, 1, 0),
    ("sample", "h", h, 0, 1),
):

    if (
        PROJECT_JSON["gps_map_caotic_{}_element_fraction_to_drop".format(w_or_h)]
        is not None
    ):

        dataframe_shape_before = dataframe.shape

        element_entropy = dataframe.apply(kraft.compute_vector_entropy, axis=apply_axis)

        dataframe.drop(
            kraft.select_series_indices(
                element_entropy,
                ">",
                fraction=PROJECT_JSON[
                    "gps_map_caotic_{}_element_fraction_to_drop".format(w_or_h)
                ],
                title={"text": PROJECT_JSON["{}_alias".format(feature_type)]},
                xaxis={"title": "Rank"},
                yaxis={"title": "Entropy in {}".format(w_or_h.title())},
            ),
            axis=drop_axis,
            inplace=True,
        )

        print(
            "{} shape: {} ==> {}.".format(
                w_or_h.title(), dataframe_shape_before, dataframe.shape
            )
        )

In [ ]:
gps_map = kraft.GPSMap(
    w=w.T,
    h=h,
    w_n_pull=PROJECT_JSON["gps_map_w_n_pull"],
    w_pull_power=PROJECT_JSON["gps_map_w_pull_power"],
    h_n_pull=PROJECT_JSON["gps_map_h_n_pull"],
    h_pull_power=PROJECT_JSON["gps_map_h_pull_power"],
    plot=False,
)

In [ ]:
for element_type, w_or_h, elements in (
    ("feature", "w", gps_map.w_elements),
    ("sample", "h", gps_map.h_elements),
):

    if elements is None:

        continue

    title = "GPS Map with {} {}s".format(
        len(elements), PROJECT_JSON["{}_alias".format(element_type)]
    )

    gps_map.plot(
        w_or_h,
        element_marker_size=PROJECT_JSON[
            "gps_map_{}_element_marker_size".format(w_or_h)
        ],
        title=title,
        html_file_path=os.path.join(PATH["{}|gps_map/".format(w_or_h)], "gps_map.html"),
    )

    if os.path.isfile(PATH["{}|cluster_x_element.tsv".format(w_or_h)]):

        element_cluster = pd.read_csv(
            PATH["{}|cluster_x_element.tsv".format(w_or_h)], sep="\t", index_col=0
        ).apply(lambda column: column.idxmax())

        cluster_n = element_cluster.value_counts()

        gps_map.set_element_label(
            w_or_h,
            element_cluster.map(
                (cluster_n.size - cluster_n.rank(method="first").astype(int)).to_dict()
            ),
            bandwidth_factor=PROJECT_JSON["gps_map_{}_bandwidth_factor".format(w_or_h)],
        )

        gps_map.plot(
            w_or_h,
            element_marker_size=PROJECT_JSON[
                "gps_map_{}_element_marker_size".format(w_or_h)
            ],
            title=title,
            html_file_path=os.path.join(
                PATH["{}|hcc|gps_map/".format(w_or_h)], "gps_map.html"
            ),
        )

In [ ]:
kraft.write_gps_map(gps_map, PATH["gps_map.pickle.gz"])

In [ ]:
for element_type, w_or_h, elements in (
    ("feature", "w", gps_map.w_elements),
    ("sample", "h", gps_map.h_elements),
):

    element_value = pd.Series(0, index=elements)

    element_value[
        element_value.index & PROJECT_JSON["{}s_to_peek".format(element_type)]
    ] = 1

    if element_value.any():

        gps_map.plot(
            w_or_h,
            annotation_x_element=element_value.to_frame().T,
            element_marker_size=PROJECT_JSON[
                "gps_map_{}_element_marker_size".format(w_or_h)
            ],
            highlight_binary=True,
            title="GPS Map with {} {}s".format(
                len(elements), PROJECT_JSON["{}_alias".format(element_type)]
            ),
            html_file_path=os.path.join(
                PATH["{}|hcc|gps_map/".format(w_or_h)], "gps_map.peek.html"
            ),
        )